In [1]:
!python --version
__author__ = "nirant.bits@gmail.com"

Python 3.6.3 :: Anaconda custom (64-bit)


# Spell Correction
One of the most frequently seen text challenges is correcting spellings. This is even all the more true when data is entered by casual human users, for instance, say shipping addresses or similar. 

Let's take an example, we want to correct `Gujrat`, `Gujart` and other minor misspelligns to  `Gujarat`.
There are several good ways to do this, depending on your dataset, and level of expertise. We discuss 2-3 popular ways, and discuss their pros and cons. 

Before I begin, we need to pay our homage to the legendary [Peter Norvig's Spell Correct](https://norvig.com/spell-correct.html). It's stil worth a read on how to _think_ about solving a problem and _exploring_ implementations. Even the way he refactors his code and writes functions is educational. 

His spell correction module is not the simplest or best way. I recommend two packages: one with a bias towards simplicity, one with a bias towards giving you all the knives, bells and whistles to try:

**[FuzzyWuzzy](https://github.com/seatgeek/fuzzywuzzy)** is easy to use. It gives a simple similarity score between two strings, capped to 100. Higher numbers mean the words are more similar. 

**[Jellyfish](https://github.com/jamesturk/jellyfish)** supports 6 edit distance functions and 4 phonetic encoding options which you can use as per your use case. 

## FuzzyWuzzy

If you have heard or used the [Levenshtein distance](https://www.wikiwand.com/en/Levenshtein_distance) (or, _edit distance_ functions in general), this package is a wrapper over the same. It uses difflib from the standard Python libs as well. 

If you want to consider implementing your own heurestics, I recommend checking out this insightful [StackOverflow Answer on Closest String Match](https://stackoverflow.com/questions/5859561/getting-the-closest-string-match)

_Tip via Wiki: Edit distance is a way of quantifying how dissimilar two strings (e.g., words) are to one another by counting the minimum number of operations required to transform one string into the other._

Let's see how we can use FuzzyWuzzy to correct our misspellings:

**Installation**

In [2]:
# import sys
# !{sys.executable} -m pip install fuzzywuzzy
# alternative for 4-10x faster computation: 
# !{sys.executable} -m pip install fuzzywuzzy[speedup]

# Collecting fuzzywuzzy
#   Downloading fuzzywuzzy-0.16.0-py2.py3-none-any.whl
# Installing collected packages: fuzzywuzzy
# Successfully installed fuzzywuzzy-0.16.0

In [3]:
from fuzzywuzzy import fuzz

In [4]:
fuzz.ratio("Electronic City Phase One", "Electronic City Phase One, Bangalore")

82

In [5]:
fuzz.partial_ratio("Electronic City Phase One", "Electronic City Phase One, Bangalore")

100

We can see how the `ratio` function is confused by the trailing “Bangalore” used in address above, but really the two strings refer to the same address/entity. This is captured by `partial_ratio`. 

Note that how both `ratio` and `partial_ratio` are sensitive to ordering of the words. This is useful for comparing addresses, which follow a rough logical order. On the other hand, if we want to compare something else like person names, it might give counterintuitive results: 



In [6]:
fuzz.ratio('Narendra Modi', 'Narendra D. Modi')

90

In [7]:
fuzz.partial_ratio('Narendra Modi', 'Narendra D. Modi')

77

Arrgh, this is not nice. Just because we had an extra `D.` token, our logic is less universal. We want something that is less order sensitive. Luckily, the authors of fuzzywuzzy kept this in mind. 

They support functions which tokenize our input on space, remove punctuations, numbers and non-ASCII characters. Then this is used to calculate similarity. Let's try that out: 

In [8]:
fuzz.token_sort_ratio('Narendra Modi', 'Narendra D. Modi')

93

In [9]:
fuzz.token_set_ratio('Narendra Modi', 'Narendra D. Modi')

100

Nice, this works perfectly for us. In case we have a list of options and we want to find the closest match(es), we can use the process module:

In [10]:
from fuzzywuzzy import process

In [11]:
query = 'Gujrat'
choices = ['Gujarat', 'Gujjar', 'Gujarat Govt.']
# Get a list of matches ordered by score, default limit to 5
print(process.extract(query, choices))

# If we want only the top one
process.extractOne(query, choices)

[('Gujarat', 92), ('Gujarat Govt.', 75), ('Gujjar', 67)]


('Gujarat', 92)

In [12]:
query = 'Banglore'
choices = ['Bangalore', 'Bengaluru']
print(process.extract(query, choices))
process.extractOne(query, choices)

[('Bangalore', 94), ('Bengaluru', 59)]


('Bangalore', 94)

In [13]:
# Let's take an example of a common search typo in online shopping:
query = 'chili'
choices = ['chilli', 'chilled', 'chilling']
print(process.extract(query, choices))
process.extractOne(query, choices)

[('chilli', 91), ('chilling', 77), ('chilled', 67)]


('chilli', 91)

## Jellyfish

This supports reasonably fast implementations of almost all popular edit distance functions (Remember? Edit distance functions tell how similar are two sequences/strings). While fuzzywuzzy supported mainly levenshtein distance, this package supports some more string comparison utilities:

- Levenshtein Distance
- Damerau-Levenshtein Distance
- Jaro Distance
- Jaro-Winkler Distance
- Match Rating Approach Comparison
- Hamming Distance

Additionally, it supports **phonetic encodings** for English. 

**Installation**

In [14]:
# import sys
# !{sys.executable} -m pip install jellyfish

# # Collecting jellyfish
# #   Downloading jellyfish-0.6.0-py3-none-any.whl
# # Installing collected packages: jellyfish
# # Successfully installed jellyfish-0.6.0

In [15]:
import jellyfish
correct_example = ('Narendra Modi', 'Narendra Modi')
damodardas_example = ('Narendra Modi', 'Narendra D. Modi')
modi_typo_example = ('Narendra Modi', 'Narendar Modi')
gujarat_typo_example = ('Gujarat', 'Gujrat')

examples = [correct_example, damodardas_example, modi_typo_example, gujarat_typo_example]

In [16]:
def calculate_distance(function, examples=examples):
    for ele in examples:
        print(f'{ele}: {function(*ele)}') 

In [17]:
calculate_distance(jellyfish.levenshtein_distance)

('Narendra Modi', 'Narendra Modi'): 0
('Narendra Modi', 'Narendra D. Modi'): 3
('Narendra Modi', 'Narendar Modi'): 2
('Gujarat', 'Gujrat'): 1


In [18]:
calculate_distance(jellyfish.damerau_levenshtein_distance)

('Narendra Modi', 'Narendra Modi'): 0
('Narendra Modi', 'Narendra D. Modi'): 3
('Narendra Modi', 'Narendar Modi'): 1
('Gujarat', 'Gujrat'): 1


In [19]:
calculate_distance(jellyfish.hamming_distance)

('Narendra Modi', 'Narendra Modi'): 0
('Narendra Modi', 'Narendra D. Modi'): 7
('Narendra Modi', 'Narendar Modi'): 2
('Gujarat', 'Gujrat'): 4


**Jaro and Jaro-Winkler** return a value of similarity - and not dissimilarity. This means that the perfect match returns 1.0 and totally unrelated match would tend to zero

In [20]:
calculate_distance(jellyfish.jaro_distance) 

('Narendra Modi', 'Narendra Modi'): 1.0
('Narendra Modi', 'Narendra D. Modi'): 0.9375
('Narendra Modi', 'Narendar Modi'): 0.9743589743589745
('Gujarat', 'Gujrat'): 0.8968253968253969


In [21]:
calculate_distance(jellyfish.jaro_winkler)

('Narendra Modi', 'Narendra Modi'): 1.0
('Narendra Modi', 'Narendra D. Modi'): 0.9625
('Narendra Modi', 'Narendar Modi'): 0.9846153846153847
('Gujarat', 'Gujrat'): 0.9277777777777778


### Phonetic Word Similarity

#### What is a phonetic encoding?

We can convert a word to a representation of its pronunciation. Ofcourse, this might vary by accents, and the conversion technique as well. 

Yet, over time 2-3 popular ways have emerged to do this. Each of these methods takes a single string and returns a coded representation. I encourage you to google each of these terms

- American Soundex (1930s) - implemented in popular database software such as PostgreSQL, MySQL, SQLite
- NYSIIS (New York State Identification and Intelligence System) - from 1970s
- Metaphone (1990s)
- Match Rating Codex (early 2000s)

Let's take a quick preview of the same: 

In [22]:
jellyfish.soundex('Jellyfish')

'J412'

In [23]:
jellyfish.nysiis('Jellyfish')

'JALYF'

In [24]:
jellyfish.metaphone('Jellyfish')

'JLFX'

In [25]:
jellyfish.match_rating_codex('Jellyfish')

'JLYFSH'

We can now use a string comparison utility that we saw earlier to compare two strings, phonetically. 

#### Metaphone + Levenshtein

For instance, `write` and `right` should have zero levenshtein distance because they are pronounced in the same way. Let's try that:

In [26]:
jellyfish.levenshtein_distance(jellyfish.metaphone('write'), jellyfish.metaphone('right'))

0

This worked as expected. Let's encapsulate this into a utility function as we did earlier. We will use two functions params now: `phonetic_func` and `distance_func`. 

In [27]:
examples+= [('write', 'right'), ('Mangalore', 'Bangalore'), ('Delhi', 'Dilli')] # adding a few examples to show how cool this is

In [28]:
def calculate_phonetic_distance(phonetic_func, distance_func, examples=examples):
    print("Word\t\tSound\t\tWord\t\t\tSound\t\tPhonetic Distance")
    for ele in examples:
        correct, typo = ele[0], ele[1]
        phonetic_correct, phonetic_typo = phonetic_func(correct), phonetic_func(typo)
        phonetic_distance = distance_func(phonetic_correct, phonetic_typo)
        print(f'{correct:<10}\t{phonetic_correct:<10}\t{typo:<20}\t{phonetic_typo:<10}\t{phonetic_distance:<10}') 
        
calculate_phonetic_distance(phonetic_func=jellyfish.metaphone, distance_func=jellyfish.levenshtein_distance)        

Word		Sound		Word			Sound		Phonetic Distance
Narendra Modi	NRNTR MT  	Narendra Modi       	NRNTR MT  	0         
Narendra Modi	NRNTR MT  	Narendra D. Modi    	NRNTR T MT	2         
Narendra Modi	NRNTR MT  	Narendar Modi       	NRNTR MT  	0         
Gujarat   	KJRT      	Gujrat              	KJRT      	0         
write     	RT        	right               	RT        	0         
Mangalore 	MNKLR     	Bangalore           	BNKLR     	1         
Delhi     	TLH       	Dilli               	TL        	1         


We notice that `Delhi` and `Dilli` are separated - which is not nice. On the other hand, `Narendra` and `Narendar` are marked as similar with zero edit distance, which is quite cool. Let's try a different technique and see how it goes:

#### American Soundex

We notice that the Soundex is aware of common similar sounding words and gives them separate phonetic encoding. This allows us to separate 'right' from 'write'. 

This will only work on American/English words though. Indian sounds such as `Narendra Modi` and `Narendra D. Modi` are now considered similar. 

In [29]:
calculate_phonetic_distance(phonetic_func=jellyfish.soundex, distance_func=jellyfish.levenshtein_distance)        

Word		Sound		Word			Sound		Phonetic Distance
Narendra Modi	N653      	Narendra Modi       	N653      	0         
Narendra Modi	N653      	Narendra D. Modi    	N653      	0         
Narendra Modi	N653      	Narendar Modi       	N653      	0         
Gujarat   	G263      	Gujrat              	G263      	0         
write     	W630      	right               	R230      	2         
Mangalore 	M524      	Bangalore           	B524      	1         
Delhi     	D400      	Dilli               	D400      	0         


#### Runtime Complexity

We now have the ability to find correct spellings of words or mark them as similar. While processing a large corpus, we can extract all unique words and compare each token against every other token. 

It would take $O(n^2)$ where $n$ = number of unique tokens in a corpus. This might make the process too slow for a large corpus. 

The alternative is to use a standard dictionary and expand the same for your corpus. If the dictionary has m unique words, this process now will be $O(m * n)$. Assuming $m << n$, this will be much faster than previous approach. 

## Updating the Original Corpus with FlashText

Using the dictionary approach, we know have a list of changes which we want to make in our corpus. We keep the original so that we can go back and apply different processing pipelines as we would like. We would want to correct these spellings in a duplicate. 

We are looking for exact word matches i.e. `Javascript` and `Java` are different entitites and each has to be treated as unique. This makes regex a great tool for our use. 

Our changes are string substitutions e.g. replace `Banglore` with `Bangalore`. 

_String Substitution_ is a common use case for standardizing multiple occurrences of the same word. For instance, you might want to replace the word with its lemma which we saw above. Or simply, case standardization. Like replacing Python3 with Python. 

_Search_ is another use case. We want to quickly check if a word exists in the corpus. E.g. we want to find out whether “Python” was mentioned in a document. 

This makes the native Python string library and regex a natural choice for us. 

### Why FlashText?

Unfortunately, it turns out that Regex is fast enough if the number of keywords to be searched and replaced is in a few 100s. 

But what about a webscale corpus with millions of documents and few thousand keywords? Regex can take several days to run over such exact searches because of its linear time complexity. How can we improve this? 


We use FlashText for this very specific use case: 

- Few million documents with few thousand keywords
- Exact keyword matches - either replace or search the presence of those keywords

Of course, there are several different solutions possible to this problem. I recommend this for it's simplicity and focus on solving one problem. It does not require us to learn new syntax or setup specific tools such as ElasticSearch. 

Comparing Flashtext vs Compiled Regex for Search: 

![Comparing Flashtext vs Compiled Regex for Search]()

Comparing FlashText vs Compiled Regex for Substitutions: 

![Comparing FlashText vs Compiled Regex for Substitutions]()

We notice that while the time taken by Regex scales almost linearly, Flashtext is relatively flat. Now we understand, that we need Flashtext for speed and scale. FlashText has seen lot of love from community. Adopters include [NLProc](https://github.com/NIHOPA/NLPre) - the NLP Preprocessing Toolkit from National Institute of Health. 

Let's get it:

**Installation**

We will first install pip the package in our conda environment. We will do from our notebook itself: 

In [30]:
import sys
!{sys.executable} -m pip install flashtext

FlashText source code is available on Github and docs are pretty easy to navigate and use. We will only consider two basic examples here. Let's figure out the syntax for finding keywords which exist in a corpus:

In [31]:
from flashtext.keyword import KeywordProcessor
keyword_processor = KeywordProcessor()
keyword_processor.add_keyword('Delhi', 'NCR') # notice we are adding tuples here
keyword_processor.add_keyword('Bombay', 'Mumbai')
keywords_found = keyword_processor.extract_keywords('I love the food in Delhi and the people in Bombay')
keywords_found
# ['NCR', 'Mumbai']

['NCR', 'Mumbai']

How about we replace them now?

In [32]:
from flashtext.keyword import KeywordProcessor
keyword_processor = KeywordProcessor()
keyword_processor.add_keyword('Delhi', 'NCR')
keyword_processor.add_keyword('Bombay', 'Mumbai')
replaced_sentence = keyword_processor.replace_keywords('I love the food in Delhi and the people in Bombay')
replaced_sentence
# 'I love the food in NCR and the people in Mumbai'

'I love the food in NCR and the people in Mumbai'

#### Limitations
FlashText only supports English for the time being. Regex can search for keywords based special characters like ^,$,*,\d,. which are not supported in FlashText. So to match partial words like word\dvec, we would still have to use regex. But it is excellent for extracting complete words like word2vec.